In [25]:
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import numpy as np

import warnings
warnings.filterwarnings("ignore")

negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [2]:
# Сбор выборки

negfeats = [" ".join(movie_reviews.words(fileids = [f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids = [f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [4]:
print("Отзывов в выборке:", len(texts))
print("Доля класса 1 в выборке:", len(posfeats)/ len(texts))

Отзывов в выборке: 2000
Доля класса 1 в выборке: 0.5


In [5]:
# заупстим CountVectorizer
vect = CountVectorizer()
Text = vect.fit_transform(texts)

In [6]:
print("Количество признаков:", len(vect.get_feature_names()))

Количество признаков: 39659


In [7]:
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression())])
clf_pipeline.fit(texts, labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
             

In [8]:
print(cross_val_score(clf_pipeline, texts, labels, scoring = 'accuracy').mean())
print(cross_val_score(clf_pipeline, texts, labels, scoring = 'roc_auc').mean())

0.8360216503929078
0.9107855118104135


In [9]:
clf = LogisticRegression()
clf.fit(Text, labels)

coeffs = clf.coef_[0]

In [10]:
print([vect.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[:2]],
     vect.vocabulary_['bad'], vect.vocabulary_['unfortunately'],
     sorted(coeffs)[:2])

print([vect.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[-2:]],
     vect.vocabulary_['great'], vect.vocabulary_['fun'],
     sorted(coeffs)[-2:])

['bad', 'unfortunately'] 2954 37056 [-0.7821764060760804, -0.6366187395273799]
['great', 'fun'] 15246 14159 [0.4442896050761445, 0.5560662269745028]


In [11]:
cv = 5
min_df = [10, 50]
def make_choice(pipeline, cv = cv, min_df = min_df):
    print("Mean:", cross_val_score(pipeline, texts, labels, scoring = 'accuracy', cv = cv).mean())
    print("Std:", cross_val_score(pipeline, texts, labels, scoring = 'accuracy', cv = cv).std())
    print("\n")
    
clf_pipeline_2 = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LogisticRegression())])
clf_pipeline_2.fit(texts, labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, inter

In [12]:
make_choice(clf_pipeline)
make_choice(clf_pipeline_2)

Mean: 0.841
Std: 0.01677796173556255


Mean: 0.8210000000000001
Std: 0.004062019202317978




In [19]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)])

for df in min_df:
    print("min_df =", df)
    print(cross_val_score(text_classifier(CountVectorizer(min_df = df), LogisticRegression()), texts, labels).mean())

print("\n")

for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), clf()), texts, labels).mean())

min_df = 10
0.8305176433918948
min_df = 50
0.811510612408816


<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8360216503929078
<class 'sklearn.svm.classes.LinearSVC'>
0.827517637397877
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.8295091498684313


In [28]:
#stop words
sw = stopwords.words('english')
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [34]:
print("NLTK stopwords:", cross_val_score(text_classifier(CountVectorizer(stop_words = sw), 
                                      LogisticRegression()), texts, labels).mean())
print("Default stopwords:", cross_val_score(text_classifier(CountVectorizer(stop_words = 'english'), 
                                      LogisticRegression()), texts, labels).mean())

print("ngram range (1, 2):", cross_val_score(text_classifier(CountVectorizer(ngram_range = (1, 2)), 
                                      LogisticRegression()), texts, labels).mean())

print("ngram range (3, 5):", cross_val_score(text_classifier(CountVectorizer(ngram_range = (3, 5), analyzer = 'char_wb'), 
                                      LogisticRegression()), texts, labels).mean())

NLTK stopwords: 0.8270171368973763
Default stopwords: 0.8345141548734363
ngram range (1, 2): 0.8375261489033944
ngram range (3, 5): 0.8205031378684073
